# 1. Data preparation
This notebook 
* downloads the used CML datasets [OpenMRG (Andersson et al. 2022)]() and [OpenRainER (Covi&Roversi 2023)](https://zenodo.org/records/10610886)
* transform the into a common data format  
* compute radar along cml

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import glob as glob
import pandas as pd
import pyproj
import gzip, tarfile
import zipfile
import shutil
import os

In [2]:
# Import OpenSense modules as submodules
import sys
import os

sys.path.append(os.path.abspath("./pycomlink/"))
sys.path.append(os.path.abspath("./poligrain/src/"))
sys.path.append(os.path.abspath("./mergeplg/src/"))
sys.path.append(os.path.abspath("./OPENSENSE_sandbox/notebooks/"))

import pycomlink as pycml 
import poligrain as plg
import mergeplg
import opensense_data_downloader_and_transformer as oddt

## 1.1 OpenMRG

#### Download OpenMRG dataset with code from [OpenSense sandbox](link) and transform to the data format standards given in [Fencl et al. 2023](https://open-research-europe.ec.europa.eu/articles/3-169).

In [3]:
# function from OpenSense sandbox 
oddt.download_andersson_2022_OpenMRG(
    local_path="data/andersson_2022_OpenMRG/", print_output=True
)

File already exists at desired location data/andersson_2022_OpenMRG/OpenMRG.zip
Not downloading!


In [4]:
# read data in 2 chunks, to save computer memory
ds_cml1 = oddt.transform_andersson_2022_OpenMRG(
    fn="data/andersson_2022_OpenMRG/OpenMRG.zip",
    path_to_extract_to="data/andersson_2022_OpenMRG/",
    time_start_end=(
        "2015-06-01",
        "2015-07-25T00:00",
    ),  
    restructure_data=True,
).resample(time = '1min').first()

ds_cml2 = oddt.transform_andersson_2022_OpenMRG(
    fn="data/andersson_2022_OpenMRG/OpenMRG.zip",
    path_to_extract_to="data/andersson_2022_OpenMRG/",
    time_start_end=(
        "2015-07-25T00:00",
        None,
    ),  
    restructure_data=True,
).resample(time = '1min').first()

c:\Users\Utente\Documents\radar_adjustment_intercomparison\OPENSENSE_sandbox\notebooks\opensense_data_downloader_and_transformer.py:302: FutureWarning: the `pandas.MultiIndex` object(s) passed as 'sublink' coordinate(s) or data variable(s) will no longer be implicitly promoted and wrapped into multiple indexed coordinates in the future (i.e., one coordinate for each multi-index level + one dimension coordinate). If you want to keep this behavior, you need to first wrap it explicitly using `mindex_coords = xarray.Coordinates.from_pandas_multiindex(mindex_obj, 'dim')` and pass it as coordinates, e.g., `xarray.Dataset(coords=mindex_coords)`, `dataset.assign_coords(mindex_coords)` or `dataarray.assign_coords(mindex_coords)`.
  ds_multindex = ds.assign_coords({'sublink':df_metadata.index})
c:\Users\Utente\Documents\radar_adjustment_intercomparison\OPENSENSE_sandbox\notebooks\opensense_data_downloader_and_transformer.py:302: FutureWarning: the `pandas.MultiIndex` object(s) passed as 'sublink

In [5]:
ds_cmls = xr.concat([ds_cml1, ds_cml2], dim = 'time').drop_duplicates(dim = 'time')

In [6]:
(
    ds_cmls.coords["site_0_x"],
    ds_cmls.coords["site_0_y"],
) = plg.spatial.project_point_coordinates(
    ds_cmls.site_0_lon, ds_cmls.site_0_lat, "EPSG:32632"
)
(
    ds_cmls.coords["site_1_x"],
    ds_cmls.coords["site_1_y"],
) = plg.spatial.project_point_coordinates(
    ds_cmls.site_1_lon, ds_cmls.site_1_lat, "EPSG:32632"
)

ds_cmls.coords["x"] = (ds_cmls.site_0_x + ds_cmls.site_1_x) / 2
ds_cmls.coords["y"] = (ds_cmls.site_0_y + ds_cmls.site_1_y) / 2

In [7]:
# Resample to 1 minute temporal resolution
ds_cmls = ds_cmls.resample(time="1min").first(skipna=True)
# change sublink identifiers names 
ds_cmls = ds_cmls.assign_coords(sublink_id = np.array(['channel1', 'channel2'])) 
# Drop first timestep (not present in gauges and radar)
ds_cmls = ds_cmls.drop_isel(time=0)

In [8]:
ds_cmls.to_netcdf('data/andersson_2022_OpenMRG/openMRG_cml.nc')

#### Merge city and smhi gauges with 15 min aggregation 

In [9]:
df_gauges_city = pd.read_csv(
    "data/andersson_2022_OpenMRG/gauges/city/CityGauges-2015JJA.csv",
    index_col=0,
    parse_dates=True,
)

df_gauges_city_metadata = pd.read_csv(
    "data/andersson_2022_OpenMRG/gauges/city/CityGauges-metadata.csv",
    index_col=0,
)

In [10]:
ds_gauges_city_org = xr.Dataset(
    data_vars=dict(
        rainfall_amount=(["id", "time"], df_gauges_city.T),
    ),
    coords=dict(
        id=df_gauges_city_metadata.index.values,
        time=df_gauges_city.index.values,
        longitude=(["id"], df_gauges_city_metadata.Longitude_DecDeg),
        latitude=(["id"], df_gauges_city_metadata.Latitude_DecDeg),
        location=(["id"], df_gauges_city_metadata.Location),
        type=(["id"], df_gauges_city_metadata.Type),
        quantization=(["id"], df_gauges_city_metadata["Resolution (mm)"]),
    ),
)
ds_gauges_city = ds_gauges_city_org.resample(time="15min", label="right").sum()

In [11]:
df_gauge_smhi = pd.read_csv(
    "data/andersson_2022_OpenMRG/gauges/smhi/GbgA-71420-2015JJA.csv",
    index_col=0,
    parse_dates=True,
)


ds_gauges_smhi = xr.Dataset(
    data_vars=dict(
        rainfall_amount=(["id", "time"], [df_gauge_smhi.Pvol_mm.values]),
    ),
    coords=dict(
        id=["SMHI"],
        time=df_gauge_smhi.index.values,
        longitude=(["id"], [11.9924]),
        latitude=(["id"], [57.7156]),
        location=(["id"], ["Goeteburg A"]),
        type=(["id"], ["15 min rainfall sum"]),
        quantization=(["id"], [0.1]),
    ),
)

In [12]:
ds_gauges_city.coords["x"], ds_gauges_city.coords["y"] = plg.spatial.project_point_coordinates(
    ds_gauges_city.longitude, ds_gauges_city.latitude, "EPSG:32632"
)

ds_gauges_smhi.coords["x"], ds_gauges_smhi.coords["y"] = (
    plg.spatial.project_point_coordinates(
        ds_gauges_smhi.longitude, ds_gauges_smhi.latitude, "EPSG:32632"
    )
)

ds_gauges_city = ds_gauges_city.rename({'longitude':'lon', 'latitude':'lat'})
ds_gauges_smhi = ds_gauges_smhi.rename({'longitude':'lon', 'latitude':'lat'})

In [13]:
ds_gauges_city.to_netcdf('data/andersson_2022_OpenMRG/gauges/openmrg_gauges_city.nc')
ds_gauges_smhi.to_netcdf('data/andersson_2022_OpenMRG/gauges/openmrg_gauges_smhi.nc')

In [14]:
ds_gauges = xr.concat([ds_gauges_city, ds_gauges_smhi], dim="id")
ds_gauges = ds_gauges.sel(time=slice(ds_cmls.time.min(),ds_cmls.time.max()))

In [15]:
ds_gauges.to_netcdf('data/andersson_2022_OpenMRG/gauges/openmrg_gauges.nc')

#### Get radar along CML data

In [16]:
ds_radar = xr.open_dataset(
    "data/andersson_2022_OpenMRG/radar/radar.nc"
)
# get rain rates from radar reflecitivity
ds_radar["dBZ"] = 0.4 * ds_radar.data - 30
ds_radar["R"] = (10 ** (ds_radar.data / 10) / 200) ** (5 / 8)
ds_radar['rainfall_amount'] = ds_radar['R'] * (5 / 60)

#add lon lat grid
x_grid, y_grid = np.meshgrid(ds_radar.x.values, ds_radar.y.values)
transformer = pyproj.Transformer.from_crs(
    "+proj=stere +lat_ts=60 +ellps=bessel +lon_0=14 +lat_0=90",
    "EPSG:4326",
    always_xy=True,
)
lon_grid, lat_grid = transformer.transform(xx=x_grid, yy=y_grid)

ds_radar.coords["lon"] = (("y", "x"), lon_grid)
ds_radar.coords["lat"] = (("y", "x"), lat_grid)

ds_radar.coords["x_grid"], ds_radar.coords["y_grid"] = plg.spatial.project_point_coordinates( 
    ds_radar.lon, ds_radar.lat, "EPSG:32632"
)

In [17]:
da_intersect_weights = plg.spatial.calc_sparse_intersect_weights_for_several_cmls(
    x1_line=ds_cmls.site_0_lon.values,
    y1_line=ds_cmls.site_0_lat.values,
    x2_line=ds_cmls.site_1_lon.values,
    y2_line=ds_cmls.site_1_lat.values,
    cml_id=ds_cmls.cml_id.values,
    x_grid=ds_radar.lon.values,
    y_grid=ds_radar.lat.values,
    grid_point_location='center',
)
da_radar_along_cmls = plg.spatial.get_grid_time_series_at_intersections(
    grid_data=ds_radar.rainfall_amount,
    intersect_weights=da_intersect_weights,
)

In [18]:
# save radar along cml data
ds_radar_along_cmls=da_radar_along_cmls.to_dataset(name='rainfall_amount')
ds_radar_along_cmls.to_netcdf('data/andersson_2022_OpenMRG/radar/radar_along_cml.nc')

In [19]:
# remove unwanted variables
ds_radar = ds_radar.drop_vars(["crs", "data", "dBZ", "R"])

# Resample to 15 min resolution from 5 min
ds_radar = ds_radar.resample(time = '15min', label='right', closed='right').mean(skipna=True)
ds_radar = ds_radar*3

# Threshold lower values
radar_zero = 0.01  # here in sum mm over 15 minutes
ds_radar["rainfall_amount"] = xr.where(
    ds_radar.rainfall_amount > radar_zero, ds_radar.rainfall_amount, 0
)

# saving radar
ds_radar.to_netcdf('data/andersson_2022_OpenMRG/radar/openmrg_rad.nc') 

## 1.2 Download OpenRainER dataset 

In [20]:
local_path = './data/covi_2024_OpenRainER/'

In [21]:
# storing to external datasource as this is big due to the radar ref
oddt.download_data_file(
    url="https://zenodo.org/api/records/14731404/files-archive",
    local_path=local_path, 
    local_file_name='files-archive.zip', print_output=True
)

File already exists at desired location ./data/covi_2024_OpenRainER/files-archive.zip
Not downloading!


In [22]:
# unzip files
with zipfile.ZipFile(local_path+'files-archive.zip') as zfile:
    zfile.extractall(local_path)

In [23]:
#untar dowloaded files
for tar_filename in os.listdir(local_path):
    if tar_filename.endswith('.tar'):
        tar_path = os.path.join(local_path, tar_filename)
        
        # Estrazione del file .tar
        with tarfile.open(tar_path, 'r') as tar:
            tar.extractall(local_path)  # Estrai tutto nella stessa directory

C:\Users\Utente\AppData\Local\Temp\ipykernel_9152\418677851.py:8: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(local_path)  # Estrai tutto nella stessa directory


In [24]:
# unzip through all files in the directory
for filename in os.listdir(local_path):
    # Check if the file has a .gz extension | select only 2 month
    if (filename.endswith('.gz')) & (('202207' in filename) | ('202208' in filename)) :
        gz_path = os.path.join(local_path, filename)
        unzipped_path = os.path.join(local_path, filename[:-3])  # Remove the .gz extension
        
        # Unzip the file
        with gzip.open(gz_path, 'rb') as f_in:
            with open(unzipped_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        
        # Delete the original .gz file 
        os.remove(gz_path)

#### Merge AWS to one dataset

In [25]:
fns = sorted(glob.glob(local_path+"/AWS_20220[7-8]*nc"))

In [26]:
ids=[]
for fn in tqdm(fns):
    tmp = xr.open_dataset(fn)
    ids.append(set(tmp.id.values))
common_ids = list(set.intersection(*ids))

rainfall=[]
for fn in tqdm(fns):
    tmp = xr.open_dataset(fn)
    tmp = tmp.sel(id=common_ids)
    rainfall.append(tmp.rainfall_amount.load())


100%|██████████| 2/2 [00:00<00:00, 39.91it/s]


In [27]:
ds_gauges = xr.concat(rainfall,dim='time')
# RG projection
ds_gauges = ds_gauges.rename({'longitude':'lon', 'latitude':'lat'})

ds_gauges.coords["x"], ds_gauges.coords["y"] = plg.spatial.project_point_coordinates(
    ds_gauges.lon, ds_gauges.lat, "EPSG:32632"
)
ds_gauges.to_netcdf(local_path+"AWS_rainfall.nc")

#### Merge cmls to one dataset

In [28]:
fns = sorted(glob.glob(local_path+"CML_20220[7-8]*"))
ids=[]
for fn in tqdm(fns):
    tmp = xr.open_dataset(fn)
    ids.append(set(tmp.cml_id.values))
common_ids = list(set.intersection(*ids))

cmls=[]
for fn in tqdm(fns):
    tmp = xr.open_dataset(fn) 
    tmp = tmp.sel(cml_id=common_ids)
    cmls.append(tmp.load())


100%|██████████| 2/2 [00:00<00:00,  2.50it/s]


In [29]:
ds_cmls = xr.concat(cmls,dim='time')

(
    ds_cmls.coords["site_0_x"],
    ds_cmls.coords["site_0_y"],
) = plg.spatial.project_point_coordinates(
    ds_cmls.site_0_lon, ds_cmls.site_0_lat, "EPSG:32632"
)
(
    ds_cmls.coords["site_1_x"],
    ds_cmls.coords["site_1_y"],
) = plg.spatial.project_point_coordinates(
    ds_cmls.site_1_lon, ds_cmls.site_1_lat, "EPSG:32632"
)

ds_cmls.coords["x"] = (ds_cmls.site_0_x + ds_cmls.site_1_x) / 2
ds_cmls.coords["y"] = (ds_cmls.site_0_y + ds_cmls.site_1_y) / 2

ds_cmls.to_netcdf(local_path + "/OpenRainER_cmls.nc")

#### Extracting radar along CML path for each month
from raw files again as its more memory efficient..

In [30]:
local_path = './data/covi_2024_OpenRainER/'
fns_cml = glob.glob(local_path+"/CML_20220[7-8]*")
fns_radar = glob.glob(local_path+"/RADrain_20220[7-8]*")

In [31]:
len(fns_cml) == len(fns_radar)

True

In [32]:
fns = sorted(glob.glob(local_path + "/CML_20220[7-8]*"))
ids=[]
for fn in tqdm(fns):
    tmp = xr.open_dataset(fn)
    ids.append(set(tmp.cml_id.values))
common_ids = list(set.intersection(*ids))

100%|██████████| 2/2 [00:00<00:00, 127.48it/s]


In [33]:
list_radar_along_cmls=[]
list_radar=[]
count=[]
for i in tqdm(range(len(fns_cml))):
    ds_cml = xr.open_dataset(fns_cml[i]).sel(cml_id=common_ids)
    ds_radar = xr.open_dataset(fns_radar[i])
    # remove unnecessary cords and variables
    ds_radar = ds_radar.drop_vars(['geo_dim','mesh_dim','mosaic'])
    #add lon lat grid
    lon_grid, lat_grid = np.meshgrid(ds_radar.lon.values, ds_radar.lat.values)
    count.append(len(ds_radar.time))
    ds_radar.coords["lon_grid"] = (("lat", "lon"), lon_grid)
    ds_radar.coords["lat_grid"] = (("lat", "lon"), lat_grid)

    ds_radar = ds_radar.rename({'lon':'x', 'lat':'y'})
    ds_radar = ds_radar.rename({'lon_grid':'lon', 'lat_grid':'lat'})

    ds_radar.coords["x_grid"], ds_radar.coords["y_grid"] = plg.spatial.project_point_coordinates( 
        ds_radar.lon, ds_radar.lat, "EPSG:32632"
        )

    list_radar.append(ds_radar)

    # calculate intersection weights
    da_intersect_weights = plg.spatial.calc_sparse_intersect_weights_for_several_cmls(
        x1_line=ds_cml.site_0_lon.values,
        y1_line=ds_cml.site_0_lat.values,
        x2_line=ds_cml.site_1_lon.values,
        y2_line=ds_cml.site_1_lat.values,
        cml_id=ds_cml.cml_id.values,
        x_grid=ds_radar.lon.values,
        y_grid=ds_radar.lat.values,
        grid_point_location='center',
    )
    # calculate rainfall along intersectons
    list_radar_along_cmls.append(plg.spatial.get_grid_time_series_at_intersections(
        grid_data=ds_radar.rainfall_amount,
        intersect_weights=da_intersect_weights,
    ))
    

100%|██████████| 2/2 [00:08<00:00,  4.08s/it]


In [34]:
ds_radar = xr.concat(list_radar,dim='time')
ds_radalong = xr.concat(list_radar_along_cmls,dim='time').to_dataset(name='rainfall_amount')

In [35]:
# fill in missing time steps
ds_radar = ds_radar.reindex({'time':pd.date_range(ds_radar.time.min().values,ds_radar.time.max().values,freq='15min')})
ds_radalong = ds_radalong.reindex({'time':pd.date_range(ds_radalong.time.min().values,ds_radalong.time.max().values,freq='15min')})

In [36]:
# Threshold lower values in radar
radar_zero = 0.01  # here in sum mm over 15 minutes
ds_radar["rainfall_amount"] = xr.where(
    ds_radar.rainfall_amount > radar_zero, ds_radar.rainfall_amount, 0
)

In [37]:
# saving both radar and radar_along_cml
ds_radar.to_netcdf(local_path + "/openrainer_radar.nc")
ds_radalong.to_netcdf(local_path + "/rad_along_cml.nc")